In [1]:
import os
import pandas as pd
import requests
from time import sleep
from urllib.parse import quote
from concurrent.futures import ThreadPoolExecutor

# Crawl data 

In [2]:
csv_path = "danh_sach_loai_ran.csv"
root_folder = "Vietnam_snake"
os.makedirs(root_folder, exist_ok=True)
df = pd.read_csv(csv_path)
MAX_IMAGES = 1000

In [3]:
def download_image(photo_url, file_path):
    try:
        img_data = requests.get(photo_url, timeout=30).content
        with open(file_path, "wb") as f:
            f.write(img_data)
        return True
    except Exception as e:
        print(f"  Lỗi tải ảnh: {e}")
        return False

In [4]:
for species in df["Tên tiếng Anh/Latinh"]:
    species_folder = os.path.join(root_folder, species.replace(" ", "_"))
    os.makedirs(species_folder, exist_ok=True)
    print(f"\nĐang crawl ảnh cho loài: {species} ...")

    page = 1
    total_images = len(os.listdir(species_folder))
    stop = False

    while not stop and total_images < MAX_IMAGES:
        url = (
            "https://api.inaturalist.org/v1/observations"
            f"?taxon_name={quote(species)}"
            "&per_page=200"
            f"&page={page}"
        )
        try:
            r = requests.get(url, timeout=30)
            r.raise_for_status()
            data = r.json()
            results = data.get("results", [])
            if not results:
                break

            with ThreadPoolExecutor(max_workers=8) as executor:
                download_jobs = []
                for obs in results:
                    for idx, p in enumerate(obs.get("photos", [])):
                        if total_images >= MAX_IMAGES:   # ✅ chặn ngay khi đạt 500
                            stop = True
                            break

                        photo_url = p.get("url")
                        if not photo_url:
                            continue
                        photo_url = photo_url.replace("square", "original")

                        file_name = f"{obs['id']}_{idx}.jpg"
                        file_path = os.path.join(species_folder, file_name)

                        if not os.path.exists(file_path):
                            download_jobs.append(executor.submit(download_image, photo_url, file_path))
                            total_images += 1   # ✅ tăng ngay khi submit task

                    if stop:
                        break

                # chờ tất cả job của vòng này xong
                for job in download_jobs:
                    job.result()

            print(f"  ✅ Trang {page}: tổng {total_images} ảnh")
            page += 1
            sleep(1)

        except Exception as e:
            print(f"❌ Lỗi với loài {species}: {e}")
            break

    print(f"🎉 Hoàn tất {species}. Tổng số ảnh: {total_images}")


Đang crawl ảnh cho loài: Achalinus ater ...
  ✅ Trang 1: tổng 3 ảnh
🎉 Hoàn tất Achalinus ater. Tổng số ảnh: 3

Đang crawl ảnh cho loài: Achalinus rufescens ...
  ✅ Trang 1: tổng 334 ảnh
🎉 Hoàn tất Achalinus rufescens. Tổng số ảnh: 334

Đang crawl ảnh cho loài: Achalinus spinalis ...
  ✅ Trang 1: tổng 174 ảnh
🎉 Hoàn tất Achalinus spinalis. Tổng số ảnh: 174

Đang crawl ảnh cho loài: Achalinus vanhoensis ...
  ✅ Trang 1: tổng 1 ảnh
🎉 Hoàn tất Achalinus vanhoensis. Tổng số ảnh: 1

Đang crawl ảnh cho loài: Acrochordus granulatus ...
  ✅ Trang 1: tổng 429 ảnh
  ✅ Trang 2: tổng 429 ảnh
🎉 Hoàn tất Acrochordus granulatus. Tổng số ảnh: 429

Đang crawl ảnh cho loài: Acrochordus javanicus ...
  ✅ Trang 1: tổng 65 ảnh
🎉 Hoàn tất Acrochordus javanicus. Tổng số ảnh: 65

Đang crawl ảnh cho loài: Ahaetulla fusca ...
  ✅ Trang 1: tổng 757 ảnh
  ✅ Trang 2: tổng 757 ảnh
🎉 Hoàn tất Ahaetulla fusca. Tổng số ảnh: 757

Đang crawl ảnh cho loài: Ahaetulla prasina ...
🎉 Hoàn tất Ahaetulla prasina. Tổng số ảnh: 

## Kiểm tra rắn không có ảnh (rắn hiếm)

In [7]:
import os

root_folder = "Vietnam_snake"
empty_folders = []
count = 0

for species_folder in os.listdir(root_folder):
    folder_path = os.path.join(root_folder, species_folder)
    if os.path.isdir(folder_path):
        # Đếm số file .jpg trong folder
        jpg_files = [f for f in os.listdir(folder_path) if f.lower().endswith(".jpg")]
        if len(jpg_files) == 0:
            empty_folders.append(species_folder)
            count += 1

print("Các folder không có ảnh:")
for folder in empty_folders:
    print(folder)

print(f"Tổng số folder không có ảnh: {count}")

Các folder không có ảnh:
Calamaria_sangi
Calamaria_gialaiensis
Lycodon_akaradaya
Opisthotropis_daovantieni
Ovophis_cf._meridionalis
Rhabdophis_homongorum
Bungarus_cf._bifasciatus
Calamaria_thanhi
Opisthotropis_cucae
Amphiesmoides_ornaticeps
Achalinus_quangi
Xenopeltis_intermedius
Trimeresurus_cf._gumprechti
Oligodon_rostralis
Plagiopholis_cf._styani
Rhabdophis_angeli
Parafimbrios_vietnamensis
Dendrelaphis_cf._pictus
Oligodon_moricei
Achalinus_zugorum
Calamaria_cf._pavimentata
Boiga_bourreti
Hypsiscopus_wettsteini
Pseudoxenodon_cf._macrops
Achalinus_emilyae
Xenopeltis_hainamensis
Opisthotropis_cf._andersonii
Rhabdophis_callichoromus
Trimeresurus_cf._stejnegeri
Calamaria_dominici
Azemiops_cf._kharini
Naja_cf._kaouthia
Achalinus_tranganensis
Colubroelaps_adleri
Calamaria_abramovi
Sinomicrurus_cf._wongii
Lycodon_poyarkovi
Calamaria_strigiventris
Argyrophis_giadinhensis
Hebius_cf._optatum
Calamaria_ingermarxorum
Achalinus_juliani
Achalinus_pingbianensis
Achalinus_timi
Lycodon_yunnanensis
Tổ

### Xoá rắn không có ảnh

In [8]:
import os

root_folder = "Vietnam_snake"

for species_folder in os.listdir(root_folder):
    folder_path = os.path.join(root_folder, species_folder)
    if os.path.isdir(folder_path):
        # Kiểm tra nếu folder không có file .jpg nào
        jpg_files = [f for f in os.listdir(folder_path) if f.lower().endswith(".jpg")]
        if len(jpg_files) == 0:
            os.rmdir(folder_path)
            print(f"Đã xóa folder rỗng: {species_folder}")

Đã xóa folder rỗng: Calamaria_sangi
Đã xóa folder rỗng: Calamaria_gialaiensis
Đã xóa folder rỗng: Lycodon_akaradaya
Đã xóa folder rỗng: Opisthotropis_daovantieni
Đã xóa folder rỗng: Ovophis_cf._meridionalis
Đã xóa folder rỗng: Rhabdophis_homongorum
Đã xóa folder rỗng: Bungarus_cf._bifasciatus
Đã xóa folder rỗng: Calamaria_thanhi
Đã xóa folder rỗng: Opisthotropis_cucae
Đã xóa folder rỗng: Amphiesmoides_ornaticeps
Đã xóa folder rỗng: Achalinus_quangi
Đã xóa folder rỗng: Xenopeltis_intermedius
Đã xóa folder rỗng: Trimeresurus_cf._gumprechti
Đã xóa folder rỗng: Oligodon_rostralis
Đã xóa folder rỗng: Plagiopholis_cf._styani
Đã xóa folder rỗng: Rhabdophis_angeli
Đã xóa folder rỗng: Parafimbrios_vietnamensis
Đã xóa folder rỗng: Dendrelaphis_cf._pictus
Đã xóa folder rỗng: Oligodon_moricei
Đã xóa folder rỗng: Achalinus_zugorum
Đã xóa folder rỗng: Calamaria_cf._pavimentata
Đã xóa folder rỗng: Boiga_bourreti
Đã xóa folder rỗng: Hypsiscopus_wettsteini
Đã xóa folder rỗng: Pseudoxenodon_cf._macrops


# Crawl bổ sung khi nhận thấy số lượng ảnh còn ít

In [2]:
import os
import pandas as pd
import requests
from time import sleep
from urllib.parse import quote
from concurrent.futures import ThreadPoolExecutor

In [3]:
csv_path = "snakes_only_on_web.csv"
root_folder = "Vietnam_snake_new"
os.makedirs(root_folder, exist_ok=True)
df = pd.read_csv(csv_path)
MAX_IMAGES = 500

In [4]:
def download_image(photo_url, file_path):
    try:
        img_data = requests.get(photo_url, timeout=30).content
        with open(file_path, "wb") as f:
            f.write(img_data)
        return True
    except Exception as e:
        print(f"  Lỗi tải ảnh: {e}")
        return False

In [5]:
for species in df["Tên tiếng Anh/Latinh"]:
    species_folder = os.path.join(root_folder, species.replace(" ", "_"))
    os.makedirs(species_folder, exist_ok=True)
    print(f"\nĐang crawl ảnh cho loài: {species} ...")

    page = 1
    total_images = len(os.listdir(species_folder))
    stop = False

    while not stop and total_images < MAX_IMAGES:
        url = (
            "https://api.inaturalist.org/v1/observations"
            f"?taxon_name={quote(species)}"
            "&per_page=200"
            f"&page={page}"
        )
        try:
            r = requests.get(url, timeout=30)
            r.raise_for_status()
            data = r.json()
            results = data.get("results", [])
            if not results:
                break

            with ThreadPoolExecutor(max_workers=8) as executor:
                download_jobs = []
                for obs in results:
                    for idx, p in enumerate(obs.get("photos", [])):
                        if total_images >= MAX_IMAGES:   # ✅ chặn ngay khi đạt 500
                            stop = True
                            break

                        photo_url = p.get("url")
                        if not photo_url:
                            continue
                        photo_url = photo_url.replace("square", "original")

                        file_name = f"{obs['id']}_{idx}.jpg"
                        file_path = os.path.join(species_folder, file_name)

                        if not os.path.exists(file_path):
                            download_jobs.append(executor.submit(download_image, photo_url, file_path))
                            total_images += 1   # ✅ tăng ngay khi submit task

                    if stop:
                        break

                # chờ tất cả job của vòng này xong
                for job in download_jobs:
                    job.result()

            print(f"  ✅ Trang {page}: tổng {total_images} ảnh")
            page += 1
            sleep(1)

        except Exception as e:
            print(f"❌ Lỗi với loài {species}: {e}")
            break

    print(f"🎉 Hoàn tất {species}. Tổng số ảnh: {total_images}")


Đang crawl ảnh cho loài: Rhabdophis nuchalis ...
  ✅ Trang 1: tổng 97 ảnh
🎉 Hoàn tất Rhabdophis nuchalis. Tổng số ảnh: 97

Đang crawl ảnh cho loài: Plagiopholis styani ...
  ✅ Trang 1: tổng 41 ảnh
🎉 Hoàn tất Plagiopholis styani. Tổng số ảnh: 41

Đang crawl ảnh cho loài: Kuatun keelback ...
  ✅ Trang 1: tổng 197 ảnh
🎉 Hoàn tất Kuatun keelback. Tổng số ảnh: 197

Đang crawl ảnh cho loài: Pope's keelback ...
  ✅ Trang 1: tổng 140 ảnh
🎉 Hoàn tất Pope's keelback. Tổng số ảnh: 140

Đang crawl ảnh cho loài: Hydrophis peronii ...
  ✅ Trang 1: tổng 143 ảnh
🎉 Hoàn tất Hydrophis peronii. Tổng số ảnh: 143

Đang crawl ảnh cho loài: Sinomicrurus macclellandi ...
  ✅ Trang 1: tổng 410 ảnh
  ✅ Trang 2: tổng 410 ảnh
🎉 Hoàn tất Sinomicrurus macclellandi. Tổng số ảnh: 410

Đang crawl ảnh cho loài: Blue-lipped sea krait ...
🎉 Hoàn tất Blue-lipped sea krait. Tổng số ảnh: 500

Đang crawl ảnh cho loài: Microcephalophis gracilis ...
  ✅ Trang 1: tổng 5 ảnh
🎉 Hoàn tất Microcephalophis gracilis. Tổng số ảnh: 5


# Resize 224px

In [1]:
from PIL import Image
import os
from concurrent.futures import ThreadPoolExecutor

src_root = "Vietnam_snake"
dst_root = "Vietnam_snake_1024px"
os.makedirs(dst_root, exist_ok=True)

def resize_image(src_img_path, dst_img_path):
    try:
        img = Image.open(src_img_path)
        img = img.resize((1024, 1024), Image.LANCZOS)
        img.save(dst_img_path)
    except Exception as e:
        print(f"Lỗi resize {src_img_path}: {e}")

for species_folder in os.listdir(src_root):
    src_folder_path = os.path.join(src_root, species_folder)
    dst_folder_path = os.path.join(dst_root, species_folder)
    if os.path.isdir(src_folder_path):
        os.makedirs(dst_folder_path, exist_ok=True)
        jobs = []
        with ThreadPoolExecutor(max_workers=16) as executor:  # Tăng số luồng nếu máy mạnh
            for file in os.listdir(src_folder_path):
                if file.lower().endswith(".jpg"):
                    src_img_path = os.path.join(src_folder_path, file)
                    dst_img_path = os.path.join(dst_folder_path, file)
                    jobs.append(executor.submit(resize_image, src_img_path, dst_img_path))
            for job in jobs:
                job.result()

Lỗi resize Vietnam_snake/Hebius_sauteri/263973752_0.jpg: cannot write mode RGBA as JPEG
Lỗi resize Vietnam_snake/Rhabdophis_nigrocinctus/242465522_1.jpg: cannot identify image file 'Vietnam_snake/Rhabdophis_nigrocinctus/242465522_1.jpg'
Lỗi resize Vietnam_snake/Rhabdophis_nigrocinctus/276976208_0.jpg: cannot identify image file 'Vietnam_snake/Rhabdophis_nigrocinctus/276976208_0.jpg'
Lỗi resize Vietnam_snake/Rhabdophis_nigrocinctus/110729523_0.jpg: cannot identify image file 'Vietnam_snake/Rhabdophis_nigrocinctus/110729523_0.jpg'
Lỗi resize Vietnam_snake/Rhabdophis_nigrocinctus/35447010_0.jpg: cannot identify image file 'Vietnam_snake/Rhabdophis_nigrocinctus/35447010_0.jpg'
Lỗi resize Vietnam_snake/Rhabdophis_nigrocinctus/30875986_0.jpg: cannot identify image file 'Vietnam_snake/Rhabdophis_nigrocinctus/30875986_0.jpg'
Lỗi resize Vietnam_snake/Rhabdophis_nigrocinctus/287187716_0.jpg: cannot identify image file 'Vietnam_snake/Rhabdophis_nigrocinctus/287187716_0.jpg'
Lỗi resize Vietnam_sna

# Crawl thêm ảnh rắn với  gbif.org

In [12]:
import os
import pandas as pd
import requests
from time import sleep
from urllib.parse import quote
from concurrent.futures import ThreadPoolExecutor

In [13]:
csv_path = "gbif_image_counts.csv"
root_folder = "Vietnam_snake_new"
os.makedirs(root_folder, exist_ok=True)
df = pd.read_csv(csv_path)
MAX_IMAGES = 500

In [14]:
def download_image(photo_url, file_path):
    try:
        img_data = requests.get(photo_url, timeout=30).content
        with open(file_path, "wb") as f:
            f.write(img_data)
        return True
    except Exception as e:
        print(f"  Lỗi tải ảnh: {e}")
        return False

In [16]:
for species in df["Species"]:
    species_folder = os.path.join(root_folder, species.replace(" ", "_"))
    os.makedirs(species_folder, exist_ok=True)
    print(f"\nĐang crawl ảnh cho loài: {species} ...")

    total_images = len(os.listdir(species_folder))
    offset = 0
    stop = False

    while not stop and total_images < MAX_IMAGES:
        url = "https://api.gbif.org/v1/occurrence/search"
        params = {
            "scientificName": species,
            "mediaType": "StillImage",
            "limit": 100,
            "offset": offset
        }
        try:
            r = requests.get(url, params=params, timeout=30)
            r.raise_for_status()
            data = r.json()
            results = data.get("results", [])
            if not results:
                break

            with ThreadPoolExecutor(max_workers=8) as executor:
                download_jobs = []
                for obs in results:
                    media_list = obs.get("media", [])
                    for idx, m in enumerate(media_list):
                        if total_images >= MAX_IMAGES:
                            stop = True
                            break
                        photo_url = m.get("identifier")
                        if not photo_url:
                            continue
                        file_name = f"{obs.get('key', 'noid')}_{idx}.jpg"
                        file_path = os.path.join(species_folder, file_name)
                        if not os.path.exists(file_path):
                            download_jobs.append(executor.submit(download_image, photo_url, file_path))
                            total_images += 1
                    if stop:
                        break
                for job in download_jobs:
                    job.result()

            print(f"  ✅ Offset {offset}: tổng {total_images} ảnh")
            offset += 100
            sleep(1)

        except Exception as e:
            print(f"❌ Lỗi với loài {species}: {e}")
            break

    print(f"🎉 Hoàn tất {species}. Tổng số ảnh: {total_images}")


Đang crawl ảnh cho loài: Bungarus slowinskii ...
  ✅ Offset 0: tổng 10 ảnh
🎉 Hoàn tất Bungarus slowinskii. Tổng số ảnh: 10

Đang crawl ảnh cho loài: Hebius deschauenseei ...
  ✅ Offset 0: tổng 16 ảnh
🎉 Hoàn tất Hebius deschauenseei. Tổng số ảnh: 16

Đang crawl ảnh cho loài: Pareas hamptoni ...
  ✅ Offset 0: tổng 55 ảnh
🎉 Hoàn tất Pareas hamptoni. Tổng số ảnh: 55

Đang crawl ảnh cho loài: Enhydris innominata ...
  ✅ Offset 0: tổng 1 ảnh
🎉 Hoàn tất Enhydris innominata. Tổng số ảnh: 1

Đang crawl ảnh cho loài: Enhydris subtaeniata ...
  Lỗi tải ảnh: HTTPSConnectionPool(host='mediaphoto.mnhn.fr', port=443): Max retries exceeded with url: /media/16383776141923B5FVyP4w4tqcxkb (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x7cf43fd89160>, 'Connection to mediaphoto.mnhn.fr timed out. (connect timeout=30)'))
  Lỗi tải ảnh: HTTPSConnectionPool(host='mediaphoto.mnhn.fr', port=443): Max retries exceeded with url: /media/1638377612178j35q6EbH50rplEEe (Caused by Conne

# Resize ảnh về 640 hoặc 1000 để thử nghiệm detect rắn phục vụ cho classification

In [1]:
from PIL import Image
import os
from concurrent.futures import ThreadPoolExecutor

src_root = "Vietnam_snake_640px_detect"
dst_root = "Vietnam_snake_224px"
os.makedirs(dst_root, exist_ok=True)

def resize_image(src_img_path, dst_img_path):
    try:
        img = Image.open(src_img_path)
        img = img.resize((224, 224), Image.LANCZOS)
        img.save(dst_img_path)
    except Exception as e:
        print(f"Lỗi resize {src_img_path}: {e}")

for species_folder in os.listdir(src_root):
    src_folder_path = os.path.join(src_root, species_folder)
    dst_folder_path = os.path.join(dst_root, species_folder)
    if os.path.isdir(src_folder_path):
        os.makedirs(dst_folder_path, exist_ok=True)
        jobs = []
        with ThreadPoolExecutor(max_workers=16) as executor:  # Tăng số luồng nếu máy mạnh
            for file in os.listdir(src_folder_path):
                if file.lower().endswith(".jpg"):
                    src_img_path = os.path.join(src_folder_path, file)
                    dst_img_path = os.path.join(dst_folder_path, file)
                    jobs.append(executor.submit(resize_image, src_img_path, dst_img_path))
            for job in jobs:
                job.result()

In [18]:
import os
import hashlib
import pandas as pd

def file_md5(file_path):
    hash_md5 = hashlib.md5()
    with open(file_path, "rb") as f:
        for chunk in iter(lambda: f.read(4096), b""):
            hash_md5.update(chunk)
    return hash_md5.hexdigest()

folder1 = "Vietnam_snake_224px"
folder2 = "Vietnam_snake_new_224px"

duplicate_records = []

species_folders = set(os.listdir(folder1)).intersection(os.listdir(folder2))

for species in species_folders:
    path1 = os.path.join(folder1, species)
    path2 = os.path.join(folder2, species)
    if os.path.isdir(path1) and os.path.isdir(path2):
        # Tạo dict hash -> filename cho folder1
        hash_dict1 = {}
        for f in os.listdir(path1):
            if f.lower().endswith('.jpg'):
                file_path = os.path.join(path1, f)
                hash1 = file_md5(file_path)
                hash_dict1[hash1] = f
        # So sánh từng ảnh ở folder2 với hash ở folder1
        for f in os.listdir(path2):
            if f.lower().endswith('.jpg'):
                file_path = os.path.join(path2, f)
                hash2 = file_md5(file_path)
                if hash2 in hash_dict1:
                    duplicate_records.append({
                        "Species": species,
                        "Image_in_224px": hash_dict1[hash2],
                        "Image_in_new_224px": f
                    })

df = pd.DataFrame(duplicate_records)
df.to_csv("duplicate_images_by_content.csv", index=False, encoding="utf-8-sig")
print("✅ Đã lưu danh sách ảnh trùng nội dung vào duplicate_images_by_content.csv")

✅ Đã lưu danh sách ảnh trùng nội dung vào duplicate_images_by_content.csv


In [19]:
import os
import hashlib

def file_md5(file_path):
    hash_md5 = hashlib.md5()
    with open(file_path, "rb") as f:
        for chunk in iter(lambda: f.read(4096), b""):
            hash_md5.update(chunk)
    return hash_md5.hexdigest()

folder1 = "Vietnam_snake_224px"
folder2 = "Vietnam_snake_new_224px"

species_folders = set(os.listdir(folder1)).intersection(os.listdir(folder2))

for species in species_folders:
    path1 = os.path.join(folder1, species)
    path2 = os.path.join(folder2, species)
    if os.path.isdir(path1) and os.path.isdir(path2):
        # Tạo set hash cho folder1
        hash_set1 = set()
        for f in os.listdir(path1):
            if f.lower().endswith('.jpg'):
                file_path = os.path.join(path1, f)
                hash1 = file_md5(file_path)
                hash_set1.add(hash1)
        # Xóa ảnh trùng hash ở folder2
        for f in os.listdir(path2):
            if f.lower().endswith('.jpg'):
                file_path = os.path.join(path2, f)
                hash2 = file_md5(file_path)
                if hash2 in hash_set1:
                    os.remove(file_path)
                    print(f"Đã xóa ảnh trùng: {species}/{f}")

Đã xóa ảnh trùng: Fimbrios_smithi/4028751012_1.jpg
Đã xóa ảnh trùng: Fimbrios_smithi/4028751012_0.jpg
Đã xóa ảnh trùng: Oligodon_lacroixi/5166960276_0.jpg
Đã xóa ảnh trùng: Oligodon_lacroixi/4527969120_0.jpg
Đã xóa ảnh trùng: Oligodon_lacroixi/4945675503_0.jpg
Đã xóa ảnh trùng: Oligodon_lacroixi/3903199153_0.jpg
Đã xóa ảnh trùng: Oligodon_lacroixi/5282054143_0.jpg
Đã xóa ảnh trùng: Lycodon_truongi/3892547423_0.jpg
Đã xóa ảnh trùng: Lycodon_truongi/5133504486_2.jpg
Đã xóa ảnh trùng: Lycodon_truongi/4982155870_2.jpg
Đã xóa ảnh trùng: Lycodon_truongi/4507660435_10.jpg
Đã xóa ảnh trùng: Lycodon_truongi/5133504486_3.jpg
Đã xóa ảnh trùng: Lycodon_truongi/4507660435_2.jpg
Đã xóa ảnh trùng: Lycodon_truongi/2856569385_0.jpg
Đã xóa ảnh trùng: Lycodon_truongi/5133504486_4.jpg
Đã xóa ảnh trùng: Lycodon_truongi/4854783310_2.jpg
Đã xóa ảnh trùng: Lycodon_truongi/4507660435_1.jpg
Đã xóa ảnh trùng: Lycodon_truongi/4982155870_0.jpg
Đã xóa ảnh trùng: Lycodon_truongi/5133504486_5.jpg
Đã xóa ảnh trùng: Ly

In [1]:
import os
import pandas as pd

root_folder = "Vietnam_snake_224px"
folders_over_500 = []

for species_folder in os.listdir(root_folder):
    folder_path = os.path.join(root_folder, species_folder)
    if os.path.isdir(folder_path):
        jpg_files = [f for f in os.listdir(folder_path) if f.lower().endswith(".jpg")]
        if len(jpg_files) >= 500:
            # Bỏ dấu _ và thay bằng khoảng trắng
            species_name = species_folder.replace("_", " ")
            folders_over_500.append({"Tên tiếng Anh/Latinh": species_name, "So_anh": len(jpg_files)})

df = pd.DataFrame(folders_over_500)
print(f"Tổng số folder có >=500 ảnh: {len(df)}")
df.to_csv("folders_over_500_images.csv", index=False, encoding="utf-8-sig")
print("✅ Đã lưu danh sách vào folders_over_500_images.csv")

Tổng số folder có >=500 ảnh: 28
✅ Đã lưu danh sách vào folders_over_500_images.csv


In [2]:
import os
import pandas as pd
import requests
from time import sleep
from urllib.parse import quote
from concurrent.futures import ThreadPoolExecutor

In [3]:
csv_path = "folders_over_500_images.csv"
root_folder = "Vietnam_snake_extra"
os.makedirs(root_folder, exist_ok=True)
df = pd.read_csv(csv_path)
MAX_IMAGES = 1000

In [4]:
def download_image(photo_url, file_path):
    try:
        img_data = requests.get(photo_url, timeout=30).content
        with open(file_path, "wb") as f:
            f.write(img_data)
        return True
    except Exception as e:
        print(f"  Lỗi tải ảnh: {e}")
        return False

In [5]:
for species in df["Tên tiếng Anh/Latinh"]:
    species_folder = os.path.join(root_folder, species.replace(" ", "_"))
    os.makedirs(species_folder, exist_ok=True)
    print(f"\nĐang crawl ảnh cho loài: {species} ...")

    page = 1
    total_images = len(os.listdir(species_folder))
    stop = False

    while not stop and total_images < MAX_IMAGES:
        url = (
            "https://api.inaturalist.org/v1/observations"
            f"?taxon_name={quote(species)}"
            "&per_page=200"
            f"&page={page}"
        )
        try:
            r = requests.get(url, timeout=30)
            r.raise_for_status()
            data = r.json()
            results = data.get("results", [])
            if not results:
                break

            with ThreadPoolExecutor(max_workers=8) as executor:
                download_jobs = []
                for obs in results:
                    for idx, p in enumerate(obs.get("photos", [])):
                        if total_images >= MAX_IMAGES:   # ✅ chặn ngay khi đạt 500
                            stop = True
                            break

                        photo_url = p.get("url")
                        if not photo_url:
                            continue
                        photo_url = photo_url.replace("square", "original")

                        file_name = f"{obs['id']}_{idx}.jpg"
                        file_path = os.path.join(species_folder, file_name)

                        if not os.path.exists(file_path):
                            download_jobs.append(executor.submit(download_image, photo_url, file_path))
                            total_images += 1   # ✅ tăng ngay khi submit task

                    if stop:
                        break

                # chờ tất cả job của vòng này xong
                for job in download_jobs:
                    job.result()

            print(f"  ✅ Trang {page}: tổng {total_images} ảnh")
            page += 1
            sleep(1)

        except Exception as e:
            print(f"❌ Lỗi với loài {species}: {e}")
            break

    print(f"🎉 Hoàn tất {species}. Tổng số ảnh: {total_images}")


Đang crawl ảnh cho loài: Lycodon ruhstrati ...
  ✅ Trang 1: tổng 371 ảnh
  ✅ Trang 2: tổng 744 ảnh
  ✅ Trang 3: tổng 1000 ảnh
🎉 Hoàn tất Lycodon ruhstrati. Tổng số ảnh: 1000

Đang crawl ảnh cho loài: Trimeresurus albolabris ...
  ✅ Trang 1: tổng 320 ảnh
  ✅ Trang 2: tổng 664 ảnh
  ✅ Trang 3: tổng 1000 ảnh
🎉 Hoàn tất Trimeresurus albolabris. Tổng số ảnh: 1000

Đang crawl ảnh cho loài: Ptyas major ...
  ✅ Trang 1: tổng 351 ảnh
  ✅ Trang 2: tổng 738 ảnh
  Lỗi tải ảnh: HTTPSConnectionPool(host='inaturalist-open-data.s3.amazonaws.com', port=443): Read timed out. (read timeout=30)
  Lỗi tải ảnh: HTTPSConnectionPool(host='inaturalist-open-data.s3.amazonaws.com', port=443): Read timed out.
  Lỗi tải ảnh: HTTPSConnectionPool(host='inaturalist-open-data.s3.amazonaws.com', port=443): Read timed out.
  Lỗi tải ảnh: HTTPSConnectionPool(host='inaturalist-open-data.s3.amazonaws.com', port=443): Read timed out.
  Lỗi tải ảnh: HTTPSConnectionPool(host='inaturalist-open-data.s3.amazonaws.com', port=443

In [6]:
from PIL import Image
import os
from concurrent.futures import ThreadPoolExecutor

src_root = "Vietnam_snake_extra"
dst_root = "Vietnam_snake_extra_224px"
os.makedirs(dst_root, exist_ok=True)

def resize_image(src_img_path, dst_img_path):
    try:
        img = Image.open(src_img_path)
        if img.size == (224, 224):
            img.save(dst_img_path)  # Chỉ copy sang folder mới
        else:
            img = img.resize((224, 224), Image.LANCZOS)
            img.save(dst_img_path)
    except Exception as e:
        print(f"Lỗi resize {src_img_path}: {e}")

for species_folder in os.listdir(src_root):
    src_folder_path = os.path.join(src_root, species_folder)
    dst_folder_path = os.path.join(dst_root, species_folder)
    if os.path.isdir(src_folder_path):
        os.makedirs(dst_folder_path, exist_ok=True)
        jobs = []
        with ThreadPoolExecutor(max_workers=16) as executor:
            for file in os.listdir(src_folder_path):
                if file.lower().endswith((".jpg", ".jpeg", ".png", ".webp")):
                    src_img_path = os.path.join(src_folder_path, file)
                    dst_img_path = os.path.join(dst_folder_path, file)
                    jobs.append(executor.submit(resize_image, src_img_path, dst_img_path))
            for job in jobs:
                job.result()

Lỗi resize Vietnam_snake_extra/Lycodon_ruhstrati/200189136_0.jpg: cannot write mode RGBA as JPEG
Lỗi resize Vietnam_snake_extra/Trimeresurus_albolabris/273673296_0.jpg: cannot write mode RGBA as JPEG
Lỗi resize Vietnam_snake_extra/Trimeresurus_albolabris/260189945_0.jpg: cannot write mode RGBA as JPEG
Lỗi resize Vietnam_snake_extra/Trimeresurus_albolabris/274485240_0.jpg: cannot write mode RGBA as JPEG
Lỗi resize Vietnam_snake_extra/Trimeresurus_albolabris/274491848_0.jpg: cannot write mode RGBA as JPEG
Lỗi resize Vietnam_snake_extra/Trimeresurus_albolabris/272865304_0.jpg: cannot write mode RGBA as JPEG
Lỗi resize Vietnam_snake_extra/Trimeresurus_albolabris/273581942_0.jpg: cannot write mode RGBA as JPEG
Lỗi resize Vietnam_snake_extra/Trimeresurus_albolabris/273698441_0.jpg: cannot write mode RGBA as JPEG
Lỗi resize Vietnam_snake_extra/Trimeresurus_albolabris/274504072_0.jpg: cannot write mode RGBA as JPEG
Lỗi resize Vietnam_snake_extra/Trimeresurus_albolabris/274330730_0.jpg: cannot 

In [7]:
import os
import shutil

src_root = "Vietnam_snake_extra_224px"
dst_root = "Vietnam_snake_224px"

for species_folder in os.listdir(src_root):
    src_folder_path = os.path.join(src_root, species_folder)
    dst_folder_path = os.path.join(dst_root, species_folder)
    if os.path.isdir(src_folder_path) and os.path.isdir(dst_folder_path):
        # Xóa folder cũ trong Vietnam_snake_224px
        shutil.rmtree(dst_folder_path)
        # Copy folder mới từ Vietnam_snake_extra_224px sang
        shutil.copytree(src_folder_path, dst_folder_path)
        print(f"Đã thay thế folder: {species_folder}")

print("✅ Đã thay thế xong các folder trùng tên.")

Đã thay thế folder: Lycodon_ruhstrati
Đã thay thế folder: Trimeresurus_albolabris
Đã thay thế folder: Ptyas_major
Đã thay thế folder: Lycodon_futsingensis
Đã thay thế folder: Oligodon_fasciolatus
Đã thay thế folder: Sibynophis_collaris
Đã thay thế folder: Lycodon_rufozonatus
Đã thay thế folder: Protobothrops_mucrosquamatus
Đã thay thế folder: Ptyas_mucosa
Đã thay thế folder: Enhydris_enhydris
Đã thay thế folder: Coelognathus_flavolineatus
Đã thay thế folder: Oligodon_formosanus
Đã thay thế folder: Chrysopelea_ornata
Đã thay thế folder: Naja_atra
Đã thay thế folder: Lycodon_capucinus
Đã thay thế folder: Trimerodytes_percarinatus
Đã thay thế folder: Indotyphlops_braminus
Đã thay thế folder: Ptyas_dhumnades
Đã thay thế folder: Ovophis_makazayazaya
Đã thay thế folder: Ahaetulla_prasina
Đã thay thế folder: Oligodon_cyclurus
Đã thay thế folder: Psammodynastes_pulverulentus
Đã thay thế folder: Fowlea_flavipunctata
Đã thay thế folder: Sibynophis_chinensis
Đã thay thế folder: Bungarus_candidus
